In [ ]:
import cv2 as cv
import numpy as np

img=np.array([[0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,0,1,0,0,0,0,0,0],
              [0,0,0,1,1,0,0,0,0,0],
              [0,0,0,1,1,1,0,0,0,0],
              [0,0,0,1,1,1,1,0,0,0],
              [0,0,0,1,1,1,1,1,0,0],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0]],dtype=np.float32)

ux=np.array([[-1,0,1]])
uy=np.array([-1,0,1]).transpose()
k=cv.getGaussianKernel(3,1)
g=np.outer(k,k.transpose())

dy=cv.filter2D(img,cv.CV_32F,uy)
dx=cv.filter2D(img,cv.CV_32F,ux)
dyy=dy*dy
dxx=dx*dx
dyx=dy*dx
gdyy=cv.filter2D(dyy,cv.CV_32F,g)
gdxx=cv.filter2D(dxx,cv.CV_32F,g)
gdyx=cv.filter2D(dyx,cv.CV_32F,g)
C=(gdyy*gdxx-gdyx*gdyx)-0.04*(gdyy+gdxx)*(gdyy+gdxx)

for j in range(1,C.shape[0]-1):		# 비최대 억제
    for i in range(1,C.shape[1]-1):
        if C[j,i]>0.1 and sum(sum(C[j,i]>C[j-1:j+2,i-1:i+2]))==8:
            img[j,i]=9			# 특징점을 원본 영상에 9로 표시
                
np.set_printoptions(precision=2)
print(dy) 
print(dx) 
print(dyy) 
print(dxx) 
print(dyx) 
print(gdyy) 
print(gdxx) 
print(gdyx) 
print(C)					# 특징 가능성 맵 
print(img)					# 특징점을 9로 표시한 원본 영상 

popping=np.zeros([160,160],np.uint8)	# 화소 확인 가능하게 16배로 확대
for j in range(0,160):
    for i in range(0,160):
        popping[j,i]=np.uint8((C[j//16,i//16]+0.06)*700)  

cv.imshow('Image Display2',popping)    
cv.waitKey()
cv.destroyAllWindows()

In [ ]:
# 03
import cv2 as cv

img = cv.imread('mot_color70.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

sift1 = cv.SIFT_create(nfeatures=2, nOctaveLayers=5, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)
kp1, des1 = sift1.detectAndCompute(gray, None)

gray1 = cv.drawKeypoints(gray, kp1, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv.imshow('SIFT1',gray1)

sift2 = cv.SIFT_create(nfeatures=4, nOctaveLayers=5, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)
kp2, des2 = sift2.detectAndCompute(gray, None)

gray2 = cv.drawKeypoints(gray, kp2, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv.imshow('SIFT2',gray2)

sift3 = cv.SIFT_create(nfeatures=8, nOctaveLayers=5, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)
kp3, des3 = sift3.detectAndCompute(gray, None)

gray3 = cv.drawKeypoints(gray, kp3, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv.imshow('SIFT3',gray3)

sift4 = cv.SIFT_create(nfeatures=16, nOctaveLayers=5, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)
kp4, des4 = sift4.detectAndCompute(gray, None)

gray4 = cv.drawKeypoints(gray, kp4, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv.imshow('SIFT4',gray4)


cv.waitKey()
cv.destroyAllWindows()

In [ ]:
# 06 옥타브 영상 만들기
import cv2 as cv
import numpy as np

# [1] 원본 이미지 읽기 (흑백)
img = cv.imread('apples.jpg', cv.IMREAD_GRAYSCALE)
if img is None:
    raise Exception('이미지 파일이 없습니다!')

# [2] 파라미터: 옥타브와 스케일 수
num_octaves = 3   # 옥타브 개수
num_scales = 5    # 옥타브당 가우시안 영상 개수

k = 2 ** (1 / num_scales)  # Scale factor between levels

# [3] 옥타브별 가우시안 & DoG 생성
for octave in range(num_octaves):
    print(f'== Octave {octave+1} ==')

    # 옥타브 이미지 (다운샘플링 적용)
    octave_img = cv.resize(
        img, 
        (img.shape[1] // (2 ** octave), img.shape[0] // (2 ** octave)),
        interpolation=cv.INTER_NEAREST
    )

    gaussians = []
    sigma = 1.6  # 초기 시그마 값 (SIFT 기본)

    # 옥타브 안에서 여러 스케일 생성
    for scale in range(num_scales):
        sigma_scale = sigma * (k ** scale)
        g = cv.GaussianBlur(octave_img, (0, 0), sigmaX=sigma_scale, sigmaY=sigma_scale)
        gaussians.append(g)

        win_name = f'O{octave+1}_G{scale+1}_σ={sigma_scale:.2f}'
        cv.imshow(win_name, g)

    # 연속 DoG 출력
    for i in range(len(gaussians) - 1):
        dog = cv.subtract(gaussians[i+1], gaussians[i])
        win_name = f'O{octave+1}_DoG{i+1}'
        cv.imshow(win_name, dog)

# [4] 대기 & 닫기
cv.waitKey(0)
cv.destroyAllWindows()


== Octave 1 ==
== Octave 2 ==
== Octave 3 ==
